### TL; DR

This notebook tries to turn the comment indexed 3 into a dataframe and then apply detoxify to that dataframe to get our final result

#### Questions that I have 

Should keep like this or immediately do the mean of each feature so that in the end our dataset will be smaller? With things as they are, if each video has 150 comments, we'll have a dataset of 150x214x64 = 2 054 400 while if we keep the idea of stocking in a csv so that we need to run the thing only once, it will have size 214x64=13 696 (much easier to work with)

In [43]:
import pandas as pd
from detoxify import Detoxify
import time

In [2]:
data_folder = './Data/' # all my data is in one folder 
path_comments='comments_glipse.csv.gz'

In [3]:
df_comments = pd.read_csv(data_folder + path_comments, compression='gzip') 
df_comments

,comments,video_id
0,"[{'id': 'UgwG1f67Ru5kL3JXmDZ4AaABAg', 'author'...",39yZGhaJay8
1,"[{'id': 'UgwDdeP0GiKI4ld_5Uh4AaABAg', 'author'...",FFm-ZpY7Fbo
2,"[{'id': 'UgySPeVqINUhtngAtbx4AaABAg', 'author'...",yfNKExa846M
3,"[{'id': 'UgyrDdhrXsWtzfv1Fbl4AaABAg', 'author'...",lCR0QTdSJfA
4,"[{'id': 'UgzHfM3U-Jx3oueGlEp4AaABAg', 'author'...",nY4EB01ldOE
...,...,...
209,"[{'id': 'UgxRhHxqrascp2Wl9XR4AaABAg', 'author'...",QAeoUItnQpE
210,"[{'id': 'UgwUu708JnCOMWgqEI54AaABAg', 'author'...",aEafDmdID70
211,"[{'id': 'UgySC3F8dJnjv0ALIW94AaABAg', 'author'...",2OVoINOkLys
212,"[{'id': 'UgwtUJyordw0yKULN6F4AaABAg', 'author'...",Ti2OUx3c6Ss


In [21]:
X=df_comments.loc[3,"comments"]
video_id=df_comments.loc[3, "video_id"]

In [54]:
# Turned 2 loops into one (no idea how to do it without a for loop)

new_list_=[] #creates a list with approximately one comment per item
n=0

for i in X.split("'id': "):
    if n!=0: #condition to avoid taking the very first i which doesn't have anything important
        [no_keep, keep]=i.split(" 'text': ")
        [text, no_keep]=keep.split(", 'likes'")
        new_list_.append(text)
    n+=1

In [57]:
df_com=pd.DataFrame(data=new_list_, columns=['comments'])

In [58]:
df_com

,comments
0,'This is one of those videos I’ll be coming ba...
1,"'Yo Jeff , Upload these kinds of vids as Podca..."
2,'BrainForce95 I second that'
3,'People who espouse socialism NEED to be slapp...
4,"'Someone resolve this mystery for me plz, I sa..."
...,...
111,'Old guys'
112,'Creepy=unattractive+tension\nSexy=attractive+...
113,"""Interest level. If she likes you, it flies. I..."
114,"""some women will find you creepy for the small..."


In [10]:
df_com.shape[0]

116

In [59]:
#issue, running time is long, 4.28 seconds for this cell

t=time.time()
Detoxify('unbiased').predict('I hate you, I love you')
print('runtime=', time.time()-t)

runtime= 4.390506744384766


In [ ]:
#is gonna run in 5 min

In [16]:
t=time.time()
N=df_com.shape[0]
list_detox_results=[]
for i in range(0, N):
    list_detox_results.append(Detoxify('unbiased').predict(df_com.loc[i, "comments"]))
print('runtime=', time.time()-t)

runtime= 299.5840299129486


In [17]:
list_detox_results

[{'toxicity': 0.0008361365,
  'severe_toxicity': 2.495947e-06,
  'obscene': 7.704473e-05,
  'identity_attack': 6.4939675e-05,
  'insult': 0.00012523835,
  'threat': 7.3382296e-05,
  'sexual_explicit': 3.5200217e-05},
 {'toxicity': 0.0013111982,
  'severe_toxicity': 2.672507e-06,
  'obscene': 0.00011780793,
  'identity_attack': 8.7914836e-05,
  'insult': 0.0002341912,
  'threat': 5.2079897e-05,
  'sexual_explicit': 5.508744e-05},
 {'toxicity': 0.0004342294,
  'severe_toxicity': 1.2943016e-06,
  'obscene': 3.108801e-05,
  'identity_attack': 7.480877e-05,
  'insult': 0.000100246514,
  'threat': 1.9219853e-05,
  'sexual_explicit': 1.090102e-05},
 {'toxicity': 0.9930239,
  'severe_toxicity': 0.00066719746,
  'obscene': 0.005575207,
  'identity_attack': 0.0050201304,
  'insult': 0.96676975,
  'threat': 0.08417095,
  'sexual_explicit': 0.0008640959},
 {'toxicity': 0.0025841566,
  'severe_toxicity': 2.6736823e-06,
  'obscene': 0.00012643274,
  'identity_attack': 0.0002035993,
  'insult': 0.000

In [30]:
toxic_df=pd.DataFrame(data=list_detox_results)

In [33]:
toxic_df.insert(0, "video_id", video_id, False) #inserts the columns video_id as the first column in the dataset

ValueError: cannot insert video_id, already exists

In [32]:
toxic_df

,video_id,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,lCR0QTdSJfA,0.000836,0.000002,0.000077,0.000065,0.000125,0.000073,0.000035
1,lCR0QTdSJfA,0.001311,0.000003,0.000118,0.000088,0.000234,0.000052,0.000055
2,lCR0QTdSJfA,0.000434,0.000001,0.000031,0.000075,0.000100,0.000019,0.000011
3,lCR0QTdSJfA,0.993024,0.000667,0.005575,0.005020,0.966770,0.084171,0.000864
4,lCR0QTdSJfA,0.002584,0.000003,0.000126,0.000204,0.000545,0.000101,0.000048
...,...,...,...,...,...,...,...,...
111,lCR0QTdSJfA,0.009181,0.000007,0.000251,0.001008,0.003268,0.000196,0.000050
112,lCR0QTdSJfA,0.069598,0.000053,0.002156,0.002092,0.002339,0.000084,0.117385
113,lCR0QTdSJfA,0.001757,0.000002,0.000064,0.000181,0.000272,0.000075,0.000035
114,lCR0QTdSJfA,0.045794,0.000073,0.000478,0.021208,0.005299,0.001618,0.004148


In [34]:
toxic_df.describe()

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
count,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,0.208492,0.008254,0.167910,0.004402,0.086535,0.001950,0.057095
std,0.374517,0.030764,0.344389,0.012114,0.196783,0.010993,0.183659
min,0.000325,0.000001,0.000021,0.000059,0.000074,0.000016,0.000011
25%,0.000514,0.000002,0.000042,0.000083,0.000113,0.000025,0.000018
50%,0.002177,0.000004,0.000166,0.000185,0.000428,0.000070,0.000071
75%,0.071047,0.000034,0.003523,0.001510,0.023568,0.000400,0.001170
max,0.994873,0.210017,0.985342,0.073049,0.966770,0.084171,0.930991


In [20]:
#if it runs very slowly, writing the data in a csv so that it ran only once and we're happy? 

#### Below I tried to use .apply to avoid using of for loop and a list but it was even slower

In [45]:
def Detoxify_df(string):
    return Detoxify('unbiased').predict(string)

In [48]:
t=time.time()

df_com['detoxify_score']=df_com.comments.apply(Detoxify_df)

print('runtime=', time.time()-t)

runtime= 437.8109531402588
